# Batch Testing Scripts
To test all of the variations of our models, we'll submit batch jobs to the clusters (using `sbatch`) to test *everything, everywhere, all at once*.

In [ ]:
"""
A simple loop to generate dead Simple Queue (dSQ) files for batch training
Usage: change the values as you desire, then run the script to generate a txt file each line of which is a job.
Then log into the cluster and execute two commands:
module load dSQ
dSQ --job-file {specified name}.txt
"""
import numpy as np
import math
# the name of your testing job, for easy reference.
name = 'DOUBLE_DYNAMO'
model_to_test = 'LEGS_Dynamo'
datasets_to_use = [ # comment out any you don't want
    # TUD Datasets
    "NCI1",
    "NCI109",
    "DD",
    "PROTEINS",
    "MUTAG",
    "PTC",
    "ENZYMES",
    "REDDIT-BINARY",
    "REDDIT-MULTI-12K",
    "IMDB-BINARY",
    "IMDB-MULTI",
    "COLLAB",
    "REDDIT-MULTI-5K",
    #Non TUD datasets
    # "ogbg-molhiv"
]
# Training specifications
transform = 'scatter_cat'
splits = [ # train-val-test.
    '8 1 1'
]
num_epochs = 1000
model_args = {}
use_cuda = True

file = open(name+'.txt','w')
for set in datasets_to_use:
    for split in splits:
        test_split = int(split[-1])
        test_seed_range = [test_split*i for i in range(int(np.ceil(10/test_split)))]
        for test_seed in test_seed_range:
            val_split = int(split[-3])
            val_seed_range = [val_split * i for i in range(int(np.ceil((10-test_split) / val_split)))]
            for val_seed in val_seed_range:
                command = f"module purge; module load miniconda; module load CUDA/10.1.105; module load cuDNN/7.6.2.24-CUDA-10.1.105; conda activate legsnet3; python argrun.py --dataset '{set}' --model '{model_to_test}' --transform '{transform}' --test_seed {test_seed} --val_seed {val_seed} --num_epochs {num_epochs} --splits {split} --out_file '{'outs/'+name+'_' + model_to_test +'_'+split.replace(' ','_')+'_'+set+'testseed'+str(test_seed)+'valseed'+str(val_seed)+'.pth'}'"
                if not use_cuda:
                    command += ' --no_cuda '
                command += '\n'
                file.write(command)
file.close()